# Mission To Mars

## Step 1 - Scraping

Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.


* Create a Jupyter Notebook file called `mission_to_mars.ipynb` and use this to complete all of your scraping and analysis tasks. The following outlines what you need to scrape.

In [ ]:
# Import Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from splinter import Browser
import time

In [ ]:
# set exec path for chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News:

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [ ]:
# set url
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# get html from current window
html = browser.html

# make soup
soup = bs(html, 'html.parser')

# get first title
news_title = soup.find('div',class_="content_title").text

# get first paragraph
news_p = soup.find('div',class_="article_teaser_body").text

# verify
print(news_title)
print(news_p)

### JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

In [ ]:
# set jpl url
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# navigate to url
browser.visit(jpl_url)

# add wait for good measure..
time.sleep(1)

# find the image
browser.find_by_id('full_image').first.click()

# another wait...
time.sleep(2)

# click details link
browser.click_link_by_partial_href('details')

# wait some more...
time.sleep(1)

# set current browser html
img_html = browser.html

# make soup
img_soup = bs(img_html,'lxml')
# print(img_soup.prettify())

# set image url
featured_image_url = 'http://jpl.nasa.gov' + img_soup.find('img',class_='main_image')['src']
#print(featured_image_url)

### Mars Weather

* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.


In [ ]:
# set mars weather twitter url
mars_wthr_url = 'https://twitter.com/marswxreport?lang=en'

# visit url
browser.visit(mars_wthr_url)

# add wait to let page load
time.sleep(1)

# grab current window html
mars_wthr_html = browser.html

# make soup
twtr_soup = bs(mars_wthr_html,'html.parser')

# print(twtr_soup.prettify())

# find tweets
tweets = twtr_soup.find_all('li',{'class':'js-stream-item stream-item stream-item '})


In [ ]:
# find first/latest weather tweet (to move past retweeted posts)
for tweet in tweets:
    # set content user
    content_u = tweet.find('span',{'class':'username u-dir u-textTruncate'}).text
    
    # set weather tweet
    mars_weather = tweet.find('p').text
    
    # see results
    print(content_u)
    print(mars_weather)
    
    # find original post and break when found
    if content_u == '@MarsWxReport':
        break


In [ ]:
# validate
print(mars_weather)

### Mars Facts

* Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [ ]:
# set url for pd scrape
mars_facts_url = 'https://space-facts.com/mars/'

In [ ]:
# read html with pandas
fact_tbls = pd.read_html(mars_facts_url)

In [ ]:
# verify
fact_tbls

In [ ]:
# check type
type(fact_tbls)

In [ ]:
# list to df
fact_df = fact_tbls[0]

# set headers
fact_df.columns = ['Attribute','Value']

# set index in place
fact_df.set_index('Attribute',inplace=True)
fact_df

In [ ]:
# set to html table
fact_df_html = fact_df.to_html()

In [ ]:
# strip new lines chars
cln_fact_df_html = fact_df_html.replace('\n','')

In [ ]:
# check table
cln_fact_df_html

In [ ]:
# write to file
fact_df.to_html('table.html')

In [ ]:
# verify
!open table.html

### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mars' hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.


In [ ]:
# mars hemisphere url
mars_hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# browser load and scrape
browser.visit(mars_hem_url)

# add wait for safe measures
time.sleep(1)

# make some soup
hem_soup = bs(browser.html,'lxml')

# find all links
thumb_list = hem_soup.find_all('div',{'class':'description'})

# verify
thumb_list



In [ ]:
# set partial link (prefix)
part_link = 'https://astrogeology.usgs.gov'

# create empty list
mars_hems = []

# loop through thumbnail list
for t in thumb_list:
    # set link text variable
    lnk = t.find('h3').text
    
    # print to see which hem is clicking...
    print(f'>>>>>searching for *{lnk}*<<<<<')
    
    # click link with text
    browser.click_link_by_partial_text(lnk)
    
    # make temp soup
    temp_soup = bs(browser.html,'lxml')
    
    # assign temp variables from soup search
    img = temp_soup.find('img','wide-image')
    div2 = temp_soup.find('div','content')
    
    # create dict
    t_dict = {'img_url':part_link + img['src'],
              'title':div2.h2.text}
    
    # check dict
    print(t_dict)
    
    # add to list
    mars_hems.append(t_dict)
    
    # go back
    browser.click_link_by_partial_text('Back')

# verify list
mars_hems


## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will import your `scrape_mars.py` script and call your `scrape` function.

  * Store the return value in Mongo as a Python dictionary.

* Create a root route `/` that will query your Mongo database and pass the mars data into an HTML template to display the data.

* Create a template HTML file called `index.html` that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

## Hints

* Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.

* Use Pymongo for CRUD applications for your database. For this homework, you can simply overwrite the existing document each time the `/scrape` url is visited and new data is obtained.

* Use Bootstrap to structure your HTML template.